# Task 2.1

In [ ]:
!pip install gurobipy



In [2]:
!pip install pandas



We make the pairwise cities in to a matrices.
That's how we can go for the TSP problem.

In [69]:
import pandas as pd

# Load data from Excel we can take any number of cities_pairwise 
df = pd.read_excel(r"C:\Users\Perpendicooler\poland_50_cities_pairwise_distances.xlsx")

# Create a list of unique city names
cities = list(set(df['From'].tolist() + df['To'].tolist()))

# Create a pivot table to organize distances
distance_matrix = df.pivot_table(values='Distance', index='From', columns='To', aggfunc='first')

# Ensure symmetry
distance_matrix = distance_matrix.add(distance_matrix.T, fill_value=0)

# Explicitly set diagonal to zeros
for city in cities:
    distance_matrix.at[city, city] = 0

# Save the distance matrix to a new Excel file
distance_matrix.to_excel(r"C:\Users\Perpendicooler\distance_matrix.xlsx")

# Display the distance matrix
print("Distance Matrix:")
print(distance_matrix)


Distance Matrix:
                     Aleksandrów Łódzki   Biały Bór      Borowa      Cegłów  \
Aleksandrów Łódzki             0.000000  284.226486  214.161892  170.596829   
Biały Bór                    284.226486    0.000000  497.199303  381.074420   
Borowa                       214.161892  497.199303    0.000000  197.793062   
Cegłów                       170.596829  381.074420  197.793062    0.000000   
Cewice                       309.114692   83.770297  512.662472  367.972080   
Chmielnik                    286.032025  567.826867   72.733736  243.407679   
Ciepielów                    169.425008  434.912965   97.223369  100.639257   
Czarków                      202.221420  453.960329  178.716826  308.436864   
Czarna Woda                  239.234630   83.112453  444.001472  307.918240   
Człopa                       257.692480  101.530931  468.903561  393.206090   
Daszyna                       38.221496  249.220473  247.980167  174.370512   
Dobre Miasto                 251.87

Created the matrix in Excel now we need to take input from the distancce_matrix and execute the TSP problem

In [70]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

# Load data from Excel, setting 'Unnamed: 0' as the index
data = pd.read_excel(r"C:\Users\Perpendicooler\distance_matrix.xlsx", index_col='Unnamed: 0')

# Extract city names
city_names = list(data.columns)
city_indices = {city: i for i, city in enumerate(city_names)}

# Extract distances
distances = {(city_indices[i], city_indices[j]): data.at[i, j] for i in city_names for j in city_names if i != j}

# Create optimization model
model_tsp = LpProblem("TSP", LpMinimize)

# Decision variables
x = {(i, j): LpVariable(name=f"x_{i}_{j}", cat='Binary') for i in city_indices.values() for j in city_indices.values() if i != j}

# Objective function
model_tsp += lpSum(distances[i, j] * x[i, j] for i in city_indices.values() for j in city_indices.values() if i != j), "Minimize Distance"

# Constraints
# Ensure that each city is visited exactly once
for i in city_indices.values():
    model_tsp += lpSum(x[i, j] for j in city_indices.values() if i != j) == 1, f"VisitOnce_{i}"

# Ensure that each city is left exactly once
for j in city_indices.values():
    model_tsp += lpSum(x[i, j] for i in city_indices.values() if i != j) == 1, f"LeaveOnce_{j}"


# Solve the model
model_tsp.solve()

# Display the results
print("Status:", LpStatus[model_tsp.status])

# Print the optimal path
optimal_path = [var for var in model_tsp.variables() if var.value() == 1]
print("Optimal Path:")
for var in sorted(optimal_path, key=lambda v: (int(v.name.split('_')[1]), int(v.name.split('_')[2]))):
    print(f"{var.name}: {var.value()}")
    
def get_city_name(index):
    return next(city for city, idx in city_indices.items() if idx == index)

# Display the optimal path
optimal_path_indices = [int(var.name.split('_')[1]) for var in optimal_path]
optimal_path_indices.append(optimal_path_indices[0])  # Add the starting city at the end to complete the loop

optimal_path_names = [get_city_name(idx) for idx in optimal_path_indices]

print("Optimal Path:")
print(" -> ".join(optimal_path_names))    


Status: Optimal
Optimal Path:
x_0_38: 1.0
x_1_9: 1.0
x_2_30: 1.0
x_3_20: 1.0
x_4_8: 1.0
x_5_34: 1.0
x_6_19: 1.0
x_7_44: 1.0
x_8_4: 1.0
x_9_1: 1.0
x_10_14: 1.0
x_11_47: 1.0
x_12_31: 1.0
x_13_17: 1.0
x_14_10: 1.0
x_15_28: 1.0
x_16_46: 1.0
x_17_13: 1.0
x_18_39: 1.0
x_19_6: 1.0
x_20_3: 1.0
x_21_15: 1.0
x_22_2: 1.0
x_23_24: 1.0
x_24_23: 1.0
x_25_33: 1.0
x_26_35: 1.0
x_27_48: 1.0
x_28_21: 1.0
x_29_43: 1.0
x_30_22: 1.0
x_31_12: 1.0
x_32_5: 1.0
x_33_25: 1.0
x_34_32: 1.0
x_35_26: 1.0
x_36_45: 1.0
x_37_42: 1.0
x_38_0: 1.0
x_39_18: 1.0
x_40_49: 1.0
x_41_11: 1.0
x_42_37: 1.0
x_43_29: 1.0
x_44_7: 1.0
x_45_36: 1.0
x_46_16: 1.0
x_47_41: 1.0
x_48_27: 1.0
x_49_40: 1.0
Optimal Path:
Aleksandrów Łódzki -> Daszyna -> Dobre Miasto -> Dwikozy -> Dziekanów Leśny -> Dąbie -> Firlej -> Godziszów -> Grójec -> Hrubieszów -> Iłża -> Biały Bór -> Jakubów -> Jastków -> Jodłówka-Wałki -> Józefów nad Wisłą -> Karczmiska -> Korczew -> Krasnopol -> Krynki -> Krzywda -> Maszkienice -> Borowa -> Niedźwiada -> Opatów -> O